# Loop 3 LB Feedback Analysis

**LB Score: 70.6600** (CV: 70.6600, gap: 0.0000)

The clean ensemble submission was successful! CV matches LB perfectly.

## Key Observations:
1. The clean ensemble (26 snapshots) scores 70.66
2. C++ optimizers (sa_v1_parallel, tree_packer_v18) cannot improve pre-optimized solutions
3. Bottom-left beam search produces worse results than baseline
4. Public datasets we can access don't have better solutions than 70.66

## Gap Analysis:
- Current: 70.66
- Target: 68.919154
- Gap: 1.74 points (2.5%)

## Next Steps:
1. Implement fractional translation in Python
2. Try backward propagation (use larger N configs for smaller N)
3. Look for better public sources or generate new solutions

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import time

# Tree polygon
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]
BASE_TREE = Polygon(zip(TX, TY))

def parse_val(v):
    if isinstance(v, str) and v.startswith('s'):
        return float(v[1:])
    return float(v)

def create_tree(x, y, deg):
    tree = affinity.rotate(BASE_TREE, deg, origin=(0, 0))
    tree = affinity.translate(tree, x, y)
    return tree

def get_bounding_box(trees):
    if not trees:
        return 0, 0, 0, 0
    min_x = min_y = float('inf')
    max_x = max_y = float('-inf')
    for tree in trees:
        b = tree.bounds
        min_x, min_y = min(min_x, b[0]), min(min_y, b[1])
        max_x, max_y = max(max_x, b[2]), max(max_y, b[3])
    return min_x, min_y, max_x, max_y

def get_side(trees):
    if not trees:
        return 0
    min_x, min_y, max_x, max_y = get_bounding_box(trees)
    return max(max_x - min_x, max_y - min_y)

def check_overlap(tree1, tree2, tolerance=1e-9):
    if tree1.intersects(tree2):
        intersection = tree1.intersection(tree2)
        if intersection.area > tolerance:
            return True
    return False

def has_any_overlap(trees, idx, tolerance=1e-9):
    for j, tree in enumerate(trees):
        if j != idx and check_overlap(trees[idx], tree, tolerance):
            return True
    return False

print('Functions loaded')

In [ ]:
# Load current submission
df = pd.read_csv('/home/submission/submission.csv')
print(f'Loaded {len(df)} rows')

# Calculate current score
total_score = 0
for n in range(1, 201):
    prefix = f'{n:03d}_'
    rows = df[df['id'].str.startswith(prefix)]
    if len(rows) != n:
        continue
    trees = []
    for _, row in rows.iterrows():
        x, y, deg = parse_val(row['x']), parse_val(row['y']), parse_val(row['deg'])
        trees.append(create_tree(x, y, deg))
    side = get_side(trees)
    total_score += side**2 / n

print(f'Current total score: {total_score:.6f}')

In [ ]:
# Implement fractional translation
def fractional_translation(trees, coords, max_iter=100):
    """Apply fractional translation to improve packing.
    
    Args:
        trees: list of Shapely polygons
        coords: list of (x, y, deg) tuples
        max_iter: maximum iterations
    
    Returns:
        improved_coords, improved_score
    """
    n = len(trees)
    best_coords = list(coords)
    best_trees = list(trees)
    best_side = get_side(best_trees)
    
    frac_steps = [0.001, 0.0005, 0.0002, 0.0001, 0.00005, 0.00002, 0.00001]
    directions = [(0, 1), (0, -1), (1, 0), (-1, 0), (1, 1), (1, -1), (-1, 1), (-1, -1)]
    
    for iteration in range(max_iter):
        improved = False
        for i in range(n):
            for step in frac_steps:
                for dx, dy in directions:
                    # Save original
                    ox, oy, odeg = best_coords[i]
                    
                    # Try new position
                    nx = ox + dx * step
                    ny = oy + dy * step
                    new_tree = create_tree(nx, ny, odeg)
                    
                    # Check overlap
                    old_tree = best_trees[i]
                    best_trees[i] = new_tree
                    
                    if not has_any_overlap(best_trees, i):
                        new_side = get_side(best_trees)
                        if new_side < best_side - 1e-12:
                            best_side = new_side
                            best_coords[i] = (nx, ny, odeg)
                            improved = True
                        else:
                            best_trees[i] = old_tree
                    else:
                        best_trees[i] = old_tree
        
        if not improved:
            break
    
    return best_coords, best_side

print('Fractional translation function defined')

In [ ]:
# Test fractional translation on a few N values
test_ns = [2, 3, 5, 10, 20]

for n in test_ns:
    prefix = f'{n:03d}_'
    rows = df[df['id'].str.startswith(prefix)]
    
    # Get current configuration
    coords = []
    trees = []
    for _, row in rows.iterrows():
        x, y, deg = parse_val(row['x']), parse_val(row['y']), parse_val(row['deg'])
        coords.append((x, y, deg))
        trees.append(create_tree(x, y, deg))
    
    original_side = get_side(trees)
    original_score = original_side**2 / n
    
    # Apply fractional translation
    start = time.time()
    new_coords, new_side = fractional_translation(trees, coords, max_iter=50)
    elapsed = time.time() - start
    
    new_score = new_side**2 / n
    improvement = original_score - new_score
    
    print(f'N={n}: original={original_score:.6f}, new={new_score:.6f}, improvement={improvement:+.6f}, time={elapsed:.1f}s')

In [ ]:
# Analyze per-N scores to find where we can improve most
per_n_scores = []
for n in range(1, 201):
    prefix = f'{n:03d}_'
    rows = df[df['id'].str.startswith(prefix)]
    if len(rows) != n:
        continue
    trees = []
    for _, row in rows.iterrows():
        x, y, deg = parse_val(row['x']), parse_val(row['y']), parse_val(row['deg'])
        trees.append(create_tree(x, y, deg))
    side = get_side(trees)
    score = side**2 / n
    
    # Theoretical minimum (tree area = 0.245625)
    tree_area = 0.245625
    min_area = n * tree_area
    min_side = np.sqrt(min_area)
    min_score = min_area / n  # = tree_area
    
    efficiency = min_score / score * 100
    per_n_scores.append({'n': n, 'score': score, 'side': side, 'efficiency': efficiency})

per_n_df = pd.DataFrame(per_n_scores)
print('Per-N score analysis:')
print(per_n_df.describe())

In [ ]:
# Find N values with lowest efficiency (most room for improvement)
print('\nN values with lowest efficiency (most room for improvement):')
worst = per_n_df.nsmallest(20, 'efficiency')
print(worst[['n', 'score', 'side', 'efficiency']].to_string())

In [ ]:
# Calculate how much improvement we need per N to reach target
target = 68.919154
current = total_score
gap = current - target

print(f'Current: {current:.6f}')
print(f'Target: {target:.6f}')
print(f'Gap: {gap:.6f} ({gap/current*100:.2f}%)')
print(f'\nNeed to reduce score by {gap:.6f} points')
print(f'Average reduction per N: {gap/200:.6f}')